In [ ]:
import os
import glob
import cv2
import imageio
import base64
import time
import ipywidgets as widgets
from IPython.display import display
import io
import numpy as np
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, lab2rgb
import csv

import torch
from torch import nn, optim
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from fastai.data.external import untar_data, URLs
from fastai.vision.learner import create_body
from torchvision.models.resnet import resnet18
from fastai.vision.models.unet import DynamicUnet

In [ ]:
seed = 123
SIZE = 256
coco_path = untar_data(URLs.COCO_SAMPLE)
coco_path = str(coco_path) + "/train_sample"
image_files = glob.glob(coco_path + "/*.jpg")
num_samples = len(image_files)
print(f"Total number of images available: {num_samples}")

paths_subset = np.array(image_files)
rand_idxs = np.random.permutation(num_samples)  # Tạo ra một hoán vị ngẫu nhiên của tất cả các chỉ số

train_idxs = rand_idxs[:int(num_samples * 0.95)]
val_idxs = rand_idxs[int(num_samples * 0.95):]

train_paths = paths_subset[train_idxs]
val_paths = paths_subset[val_idxs]

print(f"Number of training images: {len(train_paths)}")
print(f"Number of validation images: {len(val_paths)}")

class ColorizationDataset(Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE),  Image.BICUBIC),
                transforms.RandomHorizontalFlip(), # A little data augmentation!
            ])
        elif split == 'val':
            self.transforms = transforms.Resize((SIZE, SIZE),  Image.BICUBIC)

        self.split = split
        self.size = SIZE
        self.paths = paths

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        img = self.transforms(img)
        img = np.array(img)
        img_lab = rgb2lab(img).astype("float32") # Converting RGB to L*a*b
        img_lab = transforms.ToTensor()(img_lab)
        L = img_lab[[0], ...] / 50. - 1. # Between -1 and 1
        ab = img_lab[[1, 2], ...] / 110. # Between -1 and 1

        return {'L': L, 'ab': ab}

    def __len__(self):
        return len(self.paths)

def make_dataloaders(batch_size=16, n_workers=4, pin_memory=True, **kwargs):
    dataset = ColorizationDataset(**kwargs)
    dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=n_workers,
                            pin_memory=pin_memory)
    return dataloader

train_dl = make_dataloaders(paths=train_paths, split='train')
val_dl = make_dataloaders(paths=val_paths, split='val')

class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.count, self.avg, self.sum = [0.] * 3

    def update(self, val, count=1):
        self.count += count
        self.sum += count * val
        self.avg = self.sum / self.count

def create_loss_meters():
    loss_D_fake = AverageMeter()
    loss_D_real = AverageMeter()
    loss_D = AverageMeter()
    loss_G_GAN = AverageMeter()
    loss_G_L1 = AverageMeter()
    loss_G = AverageMeter()

    return {'loss_D_fake': loss_D_fake,
            'loss_D_real': loss_D_real,
            'loss_D': loss_D,
            'loss_G_GAN': loss_G_GAN,
            'loss_G_L1': loss_G_L1,
            'loss_G': loss_G}

def update_losses(model, loss_meter_dict, count):
    for loss_name, loss_meter in loss_meter_dict.items():
        loss = getattr(model, loss_name)
        loss_meter.update(loss.item(), count=count)

def lab_to_rgb(L, ab):
    """
    Takes a batch of images
    """

    L = (L + 1.) * 50.
    ab = ab * 110.
    Lab = torch.cat([L, ab], dim=1).permute(0, 2, 3, 1).cpu().numpy()
    rgb_imgs = []
    for img in Lab:
        img_rgb = lab2rgb(img)
        rgb_imgs.append(img_rgb)
    return np.stack(rgb_imgs, axis=0)

def visualize(model, data, save=True):
    model.net_G.eval()
    with torch.no_grad():
        model.setup_input(data)
        model.forward()
    model.net_G.train()
    fake_color = model.fake_color.detach()
    real_color = model.ab
    L = model.L
    fake_imgs = lab_to_rgb(L, fake_color)
    real_imgs = lab_to_rgb(L, real_color)
    fig = plt.figure(figsize=(15, 8))
    for i in range(5):
        ax = plt.subplot(3, 5, i + 1)
        ax.imshow(L[i][0].cpu(), cmap='gray')
        ax.axis("off")
        ax = plt.subplot(3, 5, i + 1 + 5)
        ax.imshow(fake_imgs[i])
        ax.axis("off")
        ax = plt.subplot(3, 5, i + 1 + 10)
        ax.imshow(real_imgs[i])
        ax.axis("off")
    plt.show()
    if save:
        fig.savefig(f"colorization_{time.time()}.png")

def log_results(loss_meter_dict):
    for loss_name, loss_meter in loss_meter_dict.items():
        print(f"{loss_name}: {loss_meter.avg:.5f}")

def build_res_unet(n_input=1, n_output=2, size=256):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    resnet18_model = resnet18(pretrained=True)  # Khởi tạo model resnet18
    body = create_body(resnet18_model, n_in=n_input, cut=-2)
    net_G = DynamicUnet(body, n_output, (size, size)).to(device)
    return net_G

class PatchDiscriminator(nn.Module):
    def __init__(self, input_c, num_filters=64, n_down=3):
        super().__init__()
        model = [self.get_layers(input_c, num_filters, norm=False)]
        model += [self.get_layers(num_filters * 2 ** i, num_filters * 2 ** (i + 1), s=1 if i == (n_down-1) else 2)
                          for i in range(n_down)] # the 'if' statement is taking care of not using
                                                  # stride of 2 for the last block in this loop
        model += [self.get_layers(num_filters * 2 ** n_down, 1, s=1, norm=False, act=False)] # Make sure to not use normalization or
                                                                                             # activation for the last layer of the model
        self.model = nn.Sequential(*model)

    def get_layers(self, ni, nf, k=4, s=2, p=1, norm=True, act=True): # when needing to make some repeatitive blocks of layers,
        layers = [nn.Conv2d(ni, nf, k, s, p, bias=not norm)]          # it's always helpful to make a separate method for that purpose
        if norm: layers += [nn.BatchNorm2d(nf)]
        if act: layers += [nn.LeakyReLU(0.2, True)]
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class GANLoss(nn.Module):
    def __init__(self, gan_mode='vanilla', real_label=1.0, fake_label=0.0):
        super().__init__()
        self.register_buffer('real_label', torch.tensor(real_label))
        self.register_buffer('fake_label', torch.tensor(fake_label))
        if gan_mode == 'vanilla':
            self.loss = nn.BCEWithLogitsLoss()
        elif gan_mode == 'lsgan':
            self.loss = nn.MSELoss()

    def get_labels(self, preds, target_is_real):
        if target_is_real:
            labels = self.real_label
        else:
            labels = self.fake_label
        return labels.expand_as(preds)

    def __call__(self, preds, target_is_real):
        labels = self.get_labels(preds, target_is_real)
        loss = self.loss(preds, labels)
        return loss

def init_weights(net, init='norm', gain=0.02):

    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and 'Conv' in classname:
            if init == 'norm':
                nn.init.normal_(m.weight.data, mean=0.0, std=gain)
            elif init == 'xavier':
                nn.init.xavier_normal_(m.weight.data, gain=gain)
            elif init == 'kaiming':
                nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')

            if hasattr(m, 'bias') and m.bias is not None:
                nn.init.constant_(m.bias.data, 0.0)
        elif 'BatchNorm2d' in classname:
            nn.init.normal_(m.weight.data, 1., gain)
            nn.init.constant_(m.bias.data, 0.)

    net.apply(init_func)
    print(f"model initialized with {init} initialization")
    return net

def init_model(model, device):
    model = model.to(device)
    model = init_weights(model)
    return model

class MainModel(nn.Module):
    def __init__(self, net_G, lr_G=2e-4, lr_D=2e-4,
                 beta1=0.5, beta2=0.999, lambda_L1=100.):
        super().__init__()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.lambda_L1 = lambda_L1
        self.net_G = net_G.to(self.device)
        self.net_D = init_model(PatchDiscriminator(input_c=3, n_down=3, num_filters=64), self.device)
        self.GANcriterion = GANLoss(gan_mode='vanilla').to(self.device)
        self.L1criterion = nn.L1Loss()
        self.opt_G = optim.Adam(self.net_G.parameters(), lr=lr_G, betas=(beta1, beta2))
        self.opt_D = optim.Adam(self.net_D.parameters(), lr=lr_D, betas=(beta1, beta2))

    def set_requires_grad(self, model, requires_grad=True):
        for p in model.parameters():
            p.requires_grad = requires_grad

    def setup_input(self, data):
        self.L = data['L'].to(self.device)
        self.ab = data['ab'].to(self.device)

    def forward(self):
        self.fake_color = self.net_G(self.L)

    def backward_D(self):
        fake_image = torch.cat([self.L, self.fake_color], dim=1)
        fake_preds = self.net_D(fake_image.detach())
        self.loss_D_fake = self.GANcriterion(fake_preds, False)
        real_image = torch.cat([self.L, self.ab], dim=1)
        real_preds = self.net_D(real_image)
        self.loss_D_real = self.GANcriterion(real_preds, True)
        self.loss_D = (self.loss_D_fake + self.loss_D_real) * 0.5
        self.loss_D.backward()

    def backward_G(self):
        fake_image = torch.cat([self.L, self.fake_color], dim=1)
        fake_preds = self.net_D(fake_image)
        self.loss_G_GAN = self.GANcriterion(fake_preds, True)
        self.loss_G_L1 = self.L1criterion(self.fake_color, self.ab) * self.lambda_L1
        self.loss_G = self.loss_G_GAN + self.loss_G_L1
        self.loss_G.backward()

    def optimize(self):
        self.forward()
        self.net_D.train()
        self.set_requires_grad(self.net_D, True)
        self.opt_D.zero_grad()
        self.backward_D()
        self.opt_D.step()

        self.net_G.train()
        self.set_requires_grad(self.net_D, False)
        self.opt_G.zero_grad()
        self.backward_G()
        self.opt_G.step()

checkpoint_dir = "/content/drive/MyDrive/Colab Notebooks/TTTN/Code"
os.makedirs(checkpoint_dir, exist_ok=True)

def train_model(model, train_dl, start_epochs, end_epochs, display_every=10, csv_filename="training_losses.csv"):
    # Initialize variables for best checkpoint tracking
    best_loss = float('inf')
    best_checkpoint_path = None
    best_epoch = start_epochs

    # Check if the CSV file exists; if not, create it and write the header
    csv_exists = os.path.isfile(csv_filename)
    if not csv_exists:
        with open(csv_filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            # Write the header
            writer.writerow(['Epoch', 'Iteration', 'loss_D_fake', 'loss_D_real', 'loss_D', 'loss_G_GAN', 'loss_G_L1', 'loss_G'])

    for e in range(start_epochs, end_epochs + 1):
        loss_meter_dict = create_loss_meters()  # Dictionary to log loss values
        i = 0  # Batch counter

        for data in tqdm(train_dl):
            model.setup_input(data)
            model.optimize()
            update_losses(model, loss_meter_dict, count=data['L'].size(0))  # Update loss values
            i += 1

            # Display results every 'display_every' iterations
            if i % display_every == 0:
                print(f"\nEpoch {e}/{end_epochs}")
                print(f"Iteration {i}/{len(train_dl)}")
                log_results(loss_meter_dict)  # Function to print loss values

                # Write the current losses to CSV
                with open(csv_filename, 'a', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    # Prepare the row data
                    row = [e, i]
                    for loss_name in ['loss_D_fake', 'loss_D_real', 'loss_D', 'loss_G_GAN', 'loss_G_L1', 'loss_G']:
                        row.append(loss_meter_dict[loss_name].avg)
                    writer.writerow(row)

        # At the end of each epoch, calculate the average loss
        avg_loss = loss_meter_dict['loss_G'].avg  # Use 'loss_G' as the total generator loss

        print(f"Epoch {e}/{end_epochs}")
        print(f"Average Loss: {avg_loss:.5f}")

        # Check if the current average loss is the best so far
        if avg_loss < best_loss:
            best_loss = avg_loss
            best_epoch = e
            checkpoint_path =  os.path.join(checkpoint_dir, f"best_checkpoint_{best_epoch}.pt")
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_G_state_dict': model.opt_G.state_dict(),
                'optimizer_D_state_dict': model.opt_D.state_dict(),
                'epoch': e,
                'loss': avg_loss,
            }, checkpoint_path)
            print(f"New best checkpoint saved at {checkpoint_path} with loss {best_loss:.5f} at epoch {best_epoch}")

            # Remove the previous best checkpoint if it exists and is different from the current one
            if best_checkpoint_path is not None and os.path.exists(best_checkpoint_path) and best_checkpoint_path != checkpoint_path:
                os.remove(best_checkpoint_path)
                print(f"Previous best checkpoint {best_checkpoint_path} has been deleted.")

            best_checkpoint_path = checkpoint_path

generator_model_path = "/kaggle/input/res18-cp100/best_checkpoint_100.pt"
net_G = build_res_unet(n_input=1, n_output=2, size=256)
res_checkpoint = torch.load(generator_model_path, map_location=device)
net_G.load_state_dict(res_checkpoint['model_state_dict'])

Total number of images available: 21837
Number of training images: 20745
Number of validation images: 1092


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 87.2MB/s]
/tmp/ipykernel_22/2060394173.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is po

<All keys matched successfully>

In [ ]:
# checkpoint_path = "/kaggle/working/cor_best_checkpoint_1.pt"
# checkpoint = torch.load(checkpoint_path, map_location=device)
# model = MainModel(net_G=net_G)
# model.load_state_dict(checkpoint['model_state_dict'])
# model.opt_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
# model.opt_D.load_state_dict(checkpoint['optimizer_D_state_dict'])
# start_epoch = checkpoint['epoch'] + 1
# train_model(model, train_dl, start_epochs=start_epoch, end_epochs=2, display_every=30)

In [ ]:
model = MainModel(net_G=net_G)
train_model(model, train_dl, start_epochs=1, end_epochs=65, display_every=30)

model initialized with norm initialization


  0%|          | 0/1297 [00:00<?, ?it/s]


Epoch 1/65
Iteration 30/1297
loss_D_fake: 0.76875
loss_D_real: 0.79572
loss_D: 0.78223
loss_G_GAN: 0.81631
loss_G_L1: 8.07021
loss_G: 8.88652

Epoch 1/65
Iteration 60/1297
loss_D_fake: 0.72243
loss_D_real: 0.74074
loss_D: 0.73158
loss_G_GAN: 0.82116
loss_G_L1: 7.96238
loss_G: 8.78354

Epoch 1/65
Iteration 90/1297
loss_D_fake: 0.68504
loss_D_real: 0.70259
loss_D: 0.69382
loss_G_GAN: 0.84835
loss_G_L1: 7.84946
loss_G: 8.69781

Epoch 1/65
Iteration 120/1297
loss_D_fake: 0.66923
loss_D_real: 0.68092
loss_D: 0.67507
loss_G_GAN: 0.87308
loss_G_L1: 7.80433
loss_G: 8.67742

Epoch 1/65
Iteration 150/1297
loss_D_fake: 0.65196
loss_D_real: 0.66235
loss_D: 0.65715
loss_G_GAN: 0.89777
loss_G_L1: 7.78240
loss_G: 8.68017

Epoch 1/65
Iteration 180/1297
loss_D_fake: 0.64793
loss_D_real: 0.65204
loss_D: 0.64999
loss_G_GAN: 0.91779
loss_G_L1: 7.76870
loss_G: 8.68650

Epoch 1/65
Iteration 210/1297
loss_D_fake: 0.63974
loss_D_real: 0.64292
loss_D: 0.64133
loss_G_GAN: 0.93345
loss_G_L1: 7.76480
loss_G: 8.6

  0%|          | 0/1297 [00:00<?, ?it/s]


Epoch 2/65
Iteration 30/1297
loss_D_fake: 0.61419
loss_D_real: 0.66662
loss_D: 0.64040
loss_G_GAN: 0.97341
loss_G_L1: 7.47585
loss_G: 8.44926

Epoch 2/65
Iteration 60/1297
loss_D_fake: 0.63634
loss_D_real: 0.65570
loss_D: 0.64602
loss_G_GAN: 0.94959
loss_G_L1: 7.41862
loss_G: 8.36822

Epoch 2/65
Iteration 90/1297
loss_D_fake: 0.62114
loss_D_real: 0.64344
loss_D: 0.63229
loss_G_GAN: 0.95712
loss_G_L1: 7.33459
loss_G: 8.29172

Epoch 2/65
Iteration 120/1297
loss_D_fake: 0.61641
loss_D_real: 0.64056
loss_D: 0.62849
loss_G_GAN: 0.97191
loss_G_L1: 7.31608
loss_G: 8.28799

Epoch 2/65
Iteration 150/1297
loss_D_fake: 0.61673
loss_D_real: 0.63955
loss_D: 0.62814
loss_G_GAN: 0.96584
loss_G_L1: 7.26964
loss_G: 8.23548

Epoch 2/65
Iteration 180/1297
loss_D_fake: 0.61691
loss_D_real: 0.64066
loss_D: 0.62878
loss_G_GAN: 0.96300
loss_G_L1: 7.28437
loss_G: 8.24738

Epoch 2/65
Iteration 210/1297
loss_D_fake: 0.61759
loss_D_real: 0.63752
loss_D: 0.62755
loss_G_GAN: 0.96150
loss_G_L1: 7.26181
loss_G: 8.2

...


Epoch 63/65
Iteration 30/1297
loss_D_fake: 0.61536
loss_D_real: 0.60646
loss_D: 0.61091
loss_G_GAN: 0.98318
loss_G_L1: 4.06330
loss_G: 5.04649

Epoch 63/65
Iteration 60/1297
loss_D_fake: 0.62327
loss_D_real: 0.62376
loss_D: 0.62351
loss_G_GAN: 0.98179
loss_G_L1: 4.07758
loss_G: 5.05937

Epoch 63/65
Iteration 90/1297
loss_D_fake: 0.62107
loss_D_real: 0.62531
loss_D: 0.62319
loss_G_GAN: 0.98013
loss_G_L1: 4.04067
loss_G: 5.02080

Epoch 63/65
Iteration 120/1297
loss_D_fake: 0.61988
loss_D_real: 0.62186
loss_D: 0.62087
loss_G_GAN: 0.98370
loss_G_L1: 4.03591
loss_G: 5.01960

Epoch 63/65
Iteration 150/1297
loss_D_fake: 0.61969
loss_D_real: 0.62166
loss_D: 0.62067
loss_G_GAN: 0.98072
loss_G_L1: 4.03430
loss_G: 5.01501

Epoch 63/65
Iteration 180/1297
loss_D_fake: 0.62489
loss_D_real: 0.62467
loss_D: 0.62478
loss_G_GAN: 0.98451
loss_G_L1: 4.03434
loss_G: 5.01885

Epoch 63/65
Iteration 210/1297
loss_D_fake: 0.62386
loss_D_real: 0.62353
loss_D: 0.62369
loss_G_GAN: 0.98103
loss_G_L1: 4.04294
loss

  0%|          | 0/1297 [00:00<?, ?it/s]


Epoch 64/65
Iteration 30/1297
loss_D_fake: 0.61737
loss_D_real: 0.61455
loss_D: 0.61596
loss_G_GAN: 0.97654
loss_G_L1: 4.04083
loss_G: 5.01737

Epoch 64/65
Iteration 60/1297
loss_D_fake: 0.63228
loss_D_real: 0.63338
loss_D: 0.63283
loss_G_GAN: 0.98135
loss_G_L1: 4.04477
loss_G: 5.02612

Epoch 64/65
Iteration 90/1297
loss_D_fake: 0.62943
loss_D_real: 0.63140
loss_D: 0.63041
loss_G_GAN: 0.97426
loss_G_L1: 4.01975
loss_G: 4.99401

Epoch 64/65
Iteration 120/1297
loss_D_fake: 0.62687
loss_D_real: 0.62709
loss_D: 0.62698
loss_G_GAN: 0.97864
loss_G_L1: 4.00903
loss_G: 4.98767

Epoch 64/65
Iteration 150/1297
loss_D_fake: 0.62504
loss_D_real: 0.62723
loss_D: 0.62614
loss_G_GAN: 0.97542
loss_G_L1: 4.00955
loss_G: 4.98497

Epoch 64/65
Iteration 180/1297
loss_D_fake: 0.62708
loss_D_real: 0.62524
loss_D: 0.62616
loss_G_GAN: 0.98055
loss_G_L1: 4.00929
loss_G: 4.98983

Epoch 64/65
Iteration 210/1297
loss_D_fake: 0.62596
loss_D_real: 0.62503
loss_D: 0.62549
loss_G_GAN: 0.98029
loss_G_L1: 4.01828
loss

  0%|          | 0/1297 [00:00<?, ?it/s]


Epoch 65/65
Iteration 30/1297
loss_D_fake: 0.61815
loss_D_real: 0.62019
loss_D: 0.61917
loss_G_GAN: 0.98203
loss_G_L1: 4.05092
loss_G: 5.03294

Epoch 65/65
Iteration 60/1297
loss_D_fake: 0.62936
loss_D_real: 0.62819
loss_D: 0.62878
loss_G_GAN: 0.97902
loss_G_L1: 4.04925
loss_G: 5.02827

Epoch 65/65
Iteration 90/1297
loss_D_fake: 0.62360
loss_D_real: 0.62589
loss_D: 0.62475
loss_G_GAN: 0.97233
loss_G_L1: 4.01287
loss_G: 4.98520

Epoch 65/65
Iteration 120/1297
loss_D_fake: 0.62002
loss_D_real: 0.62053
loss_D: 0.62027
loss_G_GAN: 0.97312
loss_G_L1: 4.01398
loss_G: 4.98709

Epoch 65/65
Iteration 150/1297
loss_D_fake: 0.62054
loss_D_real: 0.62197
loss_D: 0.62125
loss_G_GAN: 0.97297
loss_G_L1: 4.01484
loss_G: 4.98781

Epoch 65/65
Iteration 180/1297
loss_D_fake: 0.62130
loss_D_real: 0.62131
loss_D: 0.62130
loss_G_GAN: 0.97703
loss_G_L1: 4.01377
loss_G: 4.99081

Epoch 65/65
Iteration 210/1297
loss_D_fake: 0.62219
loss_D_real: 0.62218
loss_D: 0.62218
loss_G_GAN: 0.97682
loss_G_L1: 4.01949
loss